# Rekurentne nevronske mreže

V prejšnjem modulu smo obravnavali bogate semantične reprezentacije besedila. Arhitektura, ki smo jo uporabljali, zajame združeni pomen besed v stavku, vendar ne upošteva **vrstnega reda** besed, saj operacija združevanja, ki sledi vdelavam, odstrani to informacijo iz izvirnega besedila. Ker ti modeli ne morejo predstavljati vrstnega reda besed, ne morejo reševati bolj zapletenih ali dvoumnih nalog, kot sta generiranje besedila ali odgovarjanje na vprašanja.

Za zajemanje pomena zaporedja besedila bomo uporabili arhitekturo nevronske mreže, imenovano **rekurentna nevronska mreža** ali RNN. Pri uporabi RNN stavke pošiljamo skozi mrežo en token naenkrat, mreža pa ustvari neko **stanje**, ki ga nato skupaj z naslednjim tokenom ponovno pošljemo v mrežo.

![Slika, ki prikazuje primer generiranja z rekurentno nevronsko mrežo.](../../../../../translated_images/sl/rnn.27f5c29c53d727b5.webp)

Glede na vhodno zaporedje tokenov $X_0,\dots,X_n$ RNN ustvari zaporedje blokov nevronske mreže in to zaporedje trenira od začetka do konca z uporabo povratnega razširjanja napake (backpropagation). Vsak blok mreže kot vhod prejme par $(X_i,S_i)$ in kot rezultat ustvari $S_{i+1}$. Končno stanje $S_n$ ali izhod $Y_n$ gre v linearni klasifikator, da ustvari rezultat. Vsi bloki mreže si delijo iste uteži in so trenirani od začetka do konca z enim prehodom povratnega razširjanja napake.

> Zgornja slika prikazuje rekurentno nevronsko mrežo v razširjeni obliki (na levi) in v bolj kompaktni rekurentni predstavitvi (na desni). Pomembno je razumeti, da imajo vse RNN celice iste **deljive uteži**.

Ker se vektorska stanja $S_0,\dots,S_n$ prenašajo skozi mrežo, lahko RNN uči zaporedne odvisnosti med besedami. Na primer, ko se beseda *ne* pojavi nekje v zaporedju, se lahko nauči negirati določene elemente znotraj vektorskega stanja.

Vsaka RNN celica vsebuje dve matriki uteži: $W_H$ in $W_I$, ter pristranskost $b$. Na vsakem koraku RNN se glede na vhod $X_i$ in vhodno stanje $S_i$ izhodno stanje izračuna kot $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, kjer je $f$ aktivacijska funkcija (pogosto $\tanh$).

> Pri težavah, kot sta generiranje besedila (ki ga bomo obravnavali v naslednji enoti) ali strojno prevajanje, želimo dobiti tudi neko izhodno vrednost na vsakem koraku RNN. V tem primeru obstaja še ena matrika $W_O$, izhod pa se izračuna kot $Y_i=f(W_O\times S_i+b_O)$.

Poglejmo, kako nam lahko rekurentne nevronske mreže pomagajo pri klasifikaciji našega nabora novic.

> Za peskovnik okolje moramo zagnati naslednjo celico, da zagotovimo, da je potrebna knjižnica nameščena in da so podatki prednaloženi. Če izvajate lokalno, lahko naslednjo celico preskočite.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Ko treniramo velike modele, lahko razporeditev pomnilnika na GPU postane težava. Prav tako moramo eksperimentirati z različnimi velikostmi minibatch, da podatki ustrezajo pomnilniku GPU, hkrati pa je trening dovolj hiter. Če to kodo izvajate na svojem GPU stroju, lahko eksperimentirate z nastavitvijo velikosti minibatch, da pospešite trening.

> **Opomba**: Znano je, da določene različice NVidia gonilnikov po treniranju modela ne sprostijo pomnilnika. V teh zvezkih izvajamo več primerov, kar lahko povzroči izčrpanje pomnilnika v določenih nastavitvah, še posebej, če izvajate lastne eksperimente v istem zvezku. Če naletite na nenavadne napake ob začetku treniranja modela, boste morda morali znova zagnati jedro zvezka.


In [3]:
batch_size = 16
embed_size = 64

## Preprost RNN klasifikator

V primeru preprostega RNN je vsaka rekurentna enota preprosta linearna mreža, ki sprejme vhodni vektor in vektorsko stanje ter ustvari novo vektorsko stanje. V Kerasu to lahko predstavimo s plastjo `SimpleRNN`.

Čeprav lahko RNN plasti neposredno posredujemo eno-vroče kodirane tokene, to ni dobra ideja zaradi njihove visoke dimenzionalnosti. Zato bomo uporabili vgradno plast (embedding layer), da zmanjšamo dimenzionalnost vektorskih predstav besed, nato pa plast RNN in na koncu klasifikator `Dense`.

> **Opomba**: V primerih, ko dimenzionalnost ni tako visoka, na primer pri uporabi tokenizacije na ravni znakov, bi bilo smiselno eno-vroče kodirane tokene neposredno posredovati celici RNN.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Opomba:** Tukaj uporabljamo neizurjeno vgrajeno plast za enostavnost, vendar za boljše rezultate lahko uporabimo vnaprej izurjeno vgrajeno plast z uporabo Word2Vec, kot je opisano v prejšnji enoti. Dobra vaja bi bila prilagoditi to kodo za delo z vnaprej izurjenimi vgrajenimi plastmi.

Zdaj bomo izurili naš RNN. RNN-ji so na splošno precej težavni za učenje, saj je število plasti, ki sodelujejo pri povratnem razširjanju napake, zelo veliko, ko se celice RNN razširijo vzdolž dolžine zaporedja. Zato moramo izbrati manjšo stopnjo učenja in mrežo izuriti na večjem naboru podatkov, da dosežemo dobre rezultate. To lahko traja precej dolgo, zato je priporočljiva uporaba GPU-ja.

Da pospešimo proces, bomo model RNN izurili samo na naslovih novic in izpustili opise. Lahko poskusite z učenjem na opisih in preverite, ali lahko model izurite.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Opomba** da je natančnost tukaj verjetno nižja, ker treniramo samo na naslovih novic.


## Ponovno preučimo zaporedja spremenljivk

Ne pozabite, da bo plast `TextVectorization` samodejno dopolnila zaporedja spremenljive dolžine v mini seriji z zapolnitvenimi žetoni. Izkazalo se je, da ti žetoni sodelujejo tudi pri učenju, kar lahko oteži konvergenco modela.

Obstaja več pristopov, ki jih lahko uporabimo za zmanjšanje količine zapolnitve. Eden od njih je preurejanje nabora podatkov glede na dolžino zaporedja in združevanje vseh zaporedij po velikosti. To lahko storimo z uporabo funkcije `tf.data.experimental.bucket_by_sequence_length` (glejte [dokumentacijo](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Drug pristop je uporaba **maskiranja**. V Kerasu nekatere plasti podpirajo dodatni vhod, ki kaže, katere žetone je treba upoštevati pri učenju. Da vključimo maskiranje v naš model, lahko dodamo ločeno plast `Masking` ([dokumentacija](https://keras.io/api/layers/core_layers/masking/)), ali pa določimo parameter `mask_zero=True` v naši plasti `Embedding`.

> **Opomba**: To učenje bo trajalo približno 5 minut za dokončanje ene epohe na celotnem naboru podatkov. Če vam zmanjka potrpljenja, lahko učenje kadar koli prekinete. Prav tako lahko omejite količino podatkov, uporabljenih za učenje, tako da dodate klavzulo `.take(...)` po naborih podatkov `ds_train` in `ds_test`.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Zdaj, ko uporabljamo maskiranje, lahko model učimo na celotnem naboru naslovov in opisov.

> **Opomba**: Ste opazili, da smo uporabljali vektorizator, ki je bil naučen na naslovih novic, in ne na celotnem besedilu članka? To lahko povzroči, da so nekateri tokeni prezrti, zato je bolje ponovno naučiti vektorizator. Vendar pa bo to verjetno imelo le zelo majhen učinek, zato bomo zaradi enostavnosti ostali pri prej naučenem vektorizatorju.


## LSTM: Dolgoročni spomin

Ena glavnih težav RNN-jev je **izginjanje gradientov**. RNN-ji so lahko precej dolgi in imajo lahko težave pri prenosu gradientov nazaj do prve plasti mreže med povratnim razširjanjem. Ko se to zgodi, mreža ne more učiti odnosov med oddaljenimi tokeni. Eden od načinov za izogibanje tej težavi je uvedba **eksplicitnega upravljanja stanja** z uporabo **vrat**. Dve najpogostejši arhitekturi, ki uvajata vrata, sta **dolgoročni spomin** (LSTM) in **enota z zapornimi vrati** (GRU). Tukaj bomo obravnavali LSTM-je.

![Slika, ki prikazuje primer celice dolgoročnega spomina](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

LSTM mreža je organizirana podobno kot RNN, vendar se iz plasti v plast prenašata dve stanji: dejansko stanje $c$ in skriti vektor $h$. Pri vsaki enoti se skriti vektor $h_{t-1}$ združi z vhodom $x_t$, in skupaj nadzorujeta, kaj se zgodi s stanjem $c_t$ in izhodom $h_{t}$ prek **vrat**. Vsaka vrata imajo sigmoidno aktivacijo (izhod v območju $[0,1]$), ki jo lahko razumemo kot bitno masko, ko jo pomnožimo z vektorskim stanjem. LSTM-ji imajo naslednja vrata (od leve proti desni na zgornji sliki):
* **vrata za pozabo**, ki določajo, katere komponente vektorja $c_{t-1}$ moramo pozabiti in katere prenesti naprej.
* **vhodna vrata**, ki določajo, koliko informacij iz vhodnega vektorja in prejšnjega skritega vektorja je treba vključiti v vektorsko stanje.
* **izhodna vrata**, ki vzamejo novo vektorsko stanje in odločijo, katere njegove komponente bodo uporabljene za ustvarjanje novega skritega vektorja $h_t$.

Komponente stanja $c$ lahko razumemo kot zastavice, ki jih lahko vklopimo ali izklopimo. Na primer, ko v zaporedju naletimo na ime *Alice*, domnevamo, da gre za žensko, in dvignemo zastavico v stanju, ki označuje, da imamo v stavku ženski samostalnik. Ko nato naletimo na besede *and Tom*, dvignemo zastavico, ki označuje, da imamo množinski samostalnik. Tako lahko z manipulacijo stanja sledimo slovničnim lastnostim stavka.

> **Note**: Tukaj je odličen vir za razumevanje notranje strukture LSTM-jev: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) avtorja Christopherja Olaha.

Čeprav se notranja struktura LSTM celice morda zdi zapletena, Keras to implementacijo skriva znotraj plasti `LSTM`, zato je edina stvar, ki jo moramo narediti v zgornjem primeru, zamenjava povratne plasti:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## Dvosmerni in večplastni RNN-ji

V naših dosedanjih primerih so rekurentne mreže delovale od začetka zaporedja do konca. To se nam zdi naravno, saj sledi isti smeri, kot beremo ali poslušamo govor. Vendar pa je za scenarije, ki zahtevajo naključni dostop do vhodnega zaporedja, bolj smiselno izvajati rekurentne izračune v obe smeri. RNN-ji, ki omogočajo izračune v obe smeri, se imenujejo **dvosmerni** RNN-ji, in jih lahko ustvarimo tako, da rekurentni sloj ovijemo s posebnim slojem `Bidirectional`.

> **Note**: Sloj `Bidirectional` ustvari dve kopiji sloja znotraj njega in nastavi lastnost `go_backwards` ene od teh kopij na `True`, kar omogoča, da gre v nasprotno smer vzdolž zaporedja.

Rekurentne mreže, enosmerne ali dvosmerne, zajamejo vzorce znotraj zaporedja in jih shranijo v vektorske stanja ali jih vrnejo kot izhod. Tako kot pri konvolucijskih mrežah lahko zgradimo še en rekurentni sloj, ki sledi prvemu, da zajame vzorce višje ravni, zgrajene iz vzorcev nižje ravni, ki jih je izvlekel prvi sloj. To nas pripelje do pojma **večplastnega RNN-ja**, ki je sestavljen iz dveh ali več rekurentnih mrež, kjer se izhod prejšnjega sloja posreduje naslednjemu sloju kot vhod.

![Slika, ki prikazuje večplastni dolgoročno-kratkoročni pomnilniški RNN](../../../../../translated_images/sl/multi-layer-lstm.dd975e29bb2a59fe.webp)

*Slika iz [tega odličnega prispevka](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) Fernanda Lópeza.*

Keras omogoča enostavno sestavljanje teh mrež, saj morate modelu le dodati več rekurentnih slojev. Za vse sloje, razen zadnjega, moramo določiti parameter `return_sequences=True`, ker želimo, da sloj vrne vsa vmesna stanja, ne le končnega stanja rekurentnega izračuna.

Zgradimo dvoslojni dvosmerni LSTM za naš klasifikacijski problem.

> **Note** Ta koda ponovno zahteva precej časa za izvedbo, vendar nam daje najvišjo natančnost, ki smo jo do zdaj videli. Morda se torej splača počakati in preveriti rezultat.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNN-ji za druge naloge

Do zdaj smo se osredotočali na uporabo RNN-jev za razvrščanje zaporedij besedila. Vendar pa lahko obravnavajo še veliko več nalog, kot so generiranje besedila in strojno prevajanje — te naloge bomo obravnavali v naslednji enoti.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovno človeško prevajanje. Ne prevzemamo odgovornosti za morebitna nesporazuma ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
